In [1]:
from dataclasses import dataclass, field
from enum import Enum
import json
from pathlib import Path
from typing import Literal, Optional

import ezkl
import numpy as np
import onnx
import onnxruntime as ort
import torch
import torch.nn as nn
from torchinfo import summary

In [2]:
REPO_ROOT = Path().absolute().parent
REPO_ROOT

PosixPath('/home/tomaz/git/Politecnico/Projects/zkp-vml-research/my-work/ezkl')

In [3]:
MODELS_DIR = REPO_ROOT / "models"
MODELS_DIR

PosixPath('/home/tomaz/git/Politecnico/Projects/zkp-vml-research/my-work/ezkl/models')

## Define Neural Network Models

[Section 5.1](https://www.politesi.polimi.it/retrieve/ab2f9f29-9491-444a-aade-be38b88dc67d/2023_05_Cerioli_01.pdf#section.5.1)

Some Keras descriptions are left in Markdown, since the Keras option for `padding='valid'` is not currently supported by `ezkl`. For updates see [this](https://github.com/zkonduit/ezkl/pull/820) pull request.

In [4]:
class ModelAttributes(Enum):
    FNN = ("fnn", (50,))
    SMALL_CNN = ("small_cnn", (1,10,10))
    MNIST = ("mnist", (1,28,28))
    LENET5 = ("lenet5", (1,32,32))
    # VGG11 = ("vgg11", (224,224,3))

    def __init__(self, name: str, shape: tuple[int, ...]) -> None:
        self.model_name = name
        self.input_shape = shape

In [5]:
@dataclass
class Model:
    name: str
    input_shape: tuple[int, ...]
    model: nn.Module = None
    root: Optional[Path] = None
    onnx_path: Path = field(init=False)
    calibration_data_path: Path = field(init=False)
    inference_data_path: Path = field(init=False)
    output_dir: Path = field(init=False)

    def __post_init__(self):
        if self.root is None:
            self.root = REPO_ROOT
        data_dir = self.root / "data"
        self.onnx_path = self.root / "models" / f"{self.name}.onnx"
        self.calibration_data_path = data_dir / "2-calibration" / f"{self.name}.json"
        self.inference_data_path = data_dir / "3-inference" / f"{self.name}.json"
        self.output_dir = self.root / "output" / self.name

In [6]:
models = {
    attributes.model_name: Model(attributes.model_name, attributes.input_shape)
    for attributes in ModelAttributes
}
models

{'fnn': Model(name='fnn', input_shape=(50,), model=None, root=PosixPath('/home/tomaz/git/Politecnico/Projects/zkp-vml-research/my-work/ezkl'), onnx_path=PosixPath('/home/tomaz/git/Politecnico/Projects/zkp-vml-research/my-work/ezkl/models/fnn.onnx'), calibration_data_path=PosixPath('/home/tomaz/git/Politecnico/Projects/zkp-vml-research/my-work/ezkl/data/2-calibration/fnn.json'), inference_data_path=PosixPath('/home/tomaz/git/Politecnico/Projects/zkp-vml-research/my-work/ezkl/data/3-inference/fnn.json'), output_dir=PosixPath('/home/tomaz/git/Politecnico/Projects/zkp-vml-research/my-work/ezkl/output/fnn')),
 'small_cnn': Model(name='small_cnn', input_shape=(1, 10, 10), model=None, root=PosixPath('/home/tomaz/git/Politecnico/Projects/zkp-vml-research/my-work/ezkl'), onnx_path=PosixPath('/home/tomaz/git/Politecnico/Projects/zkp-vml-research/my-work/ezkl/models/small_cnn.onnx'), calibration_data_path=PosixPath('/home/tomaz/git/Politecnico/Projects/zkp-vml-research/my-work/ezkl/data/2-calibra

### Fully Connected Neural Network

```python
fnn = keras.Sequential((
    keras.layers.Input(shape=models["fnn"].input_shape),
    keras.layers.Dense(25),
    keras.layers.Dense(2),
))
fnn.compile()
fnn.summary()

models["fnn"].model = fnn
```

Since everything else is in PyTorch already, we might as well rewrite this one

In [7]:
class Fnn(nn.Module):
    def __init__(self):
        super().__init__()
        self.classifier = nn.Sequential(
            nn.Linear(in_features=models["fnn"].input_shape[0], out_features=25),
            nn.Linear(in_features=25, out_features=2),
        )

    def forward(self, x):
        x = self.classifier(x)
        return x

fnn = Fnn()
print(summary(fnn, input_size=(1, 50)))

models["fnn"].model = fnn

Layer (type:depth-idx)                   Output Shape              Param #
Fnn                                      [1, 2]                    --
├─Sequential: 1-1                        [1, 2]                    --
│    └─Linear: 2-1                       [1, 25]                   1,275
│    └─Linear: 2-2                       [1, 2]                    52
Total params: 1,327
Trainable params: 1,327
Non-trainable params: 0
Total mult-adds (M): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.01
Estimated Total Size (MB): 0.01


### Small Convolution

Using Keras:

```python
small_cnn = keras.Sequential((
    keras.layers.Input(shape=models["small_cnn"].input_shape),
    keras.layers.Conv2D(
        filters=6,
        kernel_size=3,
        data_format="channels_first",
        padding="same"
    ),
))
small_cnn.compile()
small_cnn.summary()

models["small_cnn"].model = small_cnn
```

In [8]:
class SmallCnn(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Conv2d(
            in_channels=models["small_cnn"].input_shape[0],
            out_channels=6,
            kernel_size=3,
        )

    def forward(self, x):
        x = self.conv(x)
        return x


small_cnn = SmallCnn()
print(summary(small_cnn, input_size=(1, 1, 10, 10)))

models["small_cnn"].model = small_cnn

Layer (type:depth-idx)                   Output Shape              Param #
SmallCnn                                 [1, 6, 8, 8]              --
├─Conv2d: 1-1                            [1, 6, 8, 8]              60
Total params: 60
Trainable params: 60
Non-trainable params: 0
Total mult-adds (M): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00


### Convolutional Neural Network

In [9]:
def polynomial_activation(x, a: float):
    """https://arxiv.org/abs/2011.05530"""
    return x*x + a*x

#### MNIST Conv-Net

```python
mnist = keras.Sequential((
    keras.layers.Input(shape=models["mnist"].input_shape),
    keras.layers.Conv2D(filters=4, kernel_size=3),
    # Activation multiplies values by 4 so Average Pooling becomes
    # equivalent to Sum Pooling employed in the thesis
    keras.layers.Activation(lambda x: 4*(x*x + (10**6)*x)),
    keras.layers.AvgPool2D(pool_size=2, strides=2),
    keras.layers.Conv2D(filters=8, kernel_size=3),
    # Activation multiplies values by 4 so Average Pooling becomes
    # equivalent to Sum Pooling employed in the thesis
    keras.layers.Activation(lambda x: 4*(x*x + (10**15)*x)),
    keras.layers.AvgPool2D(pool_size=2, strides=2),
    keras.layers.Flatten(),
    keras.layers.Dense(10),
))
mnist.compile()
mnist.summary()

models["mnist"].model = mnist
```

In [10]:
class Mnist(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(
            in_channels=models["mnist"].input_shape[0],
            out_channels=4,
            kernel_size=3,
        )
        self.conv2 = nn.Conv2d(in_channels=4, out_channels=8, kernel_size=3)
        self.pool = nn.AvgPool2d(kernel_size=2, stride=2)
        self.dense = nn.Linear(200, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.conv1(x)
        x = polynomial_activation(x, 1)
        # SumPool: AvgPool multiplied by number of elements (4)
        x = self.pool(x) * 4

        x = self.conv2(x)
        x = polynomial_activation(x, 1)
        # SumPool: AvgPool multiplied by number of elements (4)
        x = self.pool(x) * 4

        x = x.flatten(start_dim=1)
        x = self.dense(x)

        return x


mnist = Mnist()
print(summary(mnist, input_size=(1, 1, 28, 28)))

models["mnist"].model = mnist

Layer (type:depth-idx)                   Output Shape              Param #
Mnist                                    [1, 10]                   --
├─Conv2d: 1-1                            [1, 4, 26, 26]            40
├─AvgPool2d: 1-2                         [1, 4, 13, 13]            --
├─Conv2d: 1-3                            [1, 8, 11, 11]            296
├─AvgPool2d: 1-4                         [1, 8, 5, 5]              --
├─Linear: 1-5                            [1, 10]                   2,010
Total params: 2,346
Trainable params: 2,346
Non-trainable params: 0
Total mult-adds (M): 0.06
Input size (MB): 0.00
Forward/backward pass size (MB): 0.03
Params size (MB): 0.01
Estimated Total Size (MB): 0.04


#### LeNet5

```python
lenet5 = keras.Sequential((
    keras.layers.Input(shape=models["lenet5"].input_shape),
    keras.layers.Conv2D(filters=6, kernel_size=5),
    keras.layers.Activation(lambda x: x*x + (10**6)*x),
    keras.layers.AvgPool2D(pool_size=2, strides=2),
    keras.layers.Conv2D(filters=16, kernel_size=5),
    keras.layers.Activation(lambda x: x*x + (10**15)*x),
    keras.layers.AvgPool2D(pool_size=2, strides=2),
    keras.layers.Flatten(),
    keras.layers.Dense(120),
    keras.layers.Dense(84),
    keras.layers.Dense(10),
))
lenet5.compile()
lenet5.summary()

models["lenet5"].model = lenet5
```

In [11]:
class Lenet5(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(
            in_channels=models["lenet5"].input_shape[0],
            out_channels=6,
            kernel_size=5,
        )
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
        self.pool = nn.AvgPool2d(kernel_size=2, stride=2)
        self.classifier = nn.Sequential(
            nn.Linear(400, 120),
            nn.Linear(120, 84),
            nn.Linear(84, 10),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # Feature extraction
        x = self.conv1(x)
        x = polynomial_activation(x, 1)
        x = self.pool(x)

        x = self.conv2(x)
        x = polynomial_activation(x, 1)
        x = self.pool(x)

        x = x.flatten(start_dim=1)

        x = self.classifier(x)

        return x

torch.manual_seed(0)
lenet5 = Lenet5()
print(summary(lenet5, (1, 1, 32, 32)))

models["lenet5"].model = lenet5

Layer (type:depth-idx)                   Output Shape              Param #
Lenet5                                   [1, 10]                   --
├─Conv2d: 1-1                            [1, 6, 28, 28]            156
├─AvgPool2d: 1-2                         [1, 6, 14, 14]            --
├─Conv2d: 1-3                            [1, 16, 10, 10]           2,416
├─AvgPool2d: 1-4                         [1, 16, 5, 5]             --
├─Sequential: 1-5                        [1, 10]                   --
│    └─Linear: 2-1                       [1, 120]                  48,120
│    └─Linear: 2-2                       [1, 84]                   10,164
│    └─Linear: 2-3                       [1, 10]                   850
Total params: 61,706
Trainable params: 61,706
Non-trainable params: 0
Total mult-adds (M): 0.42
Input size (MB): 0.00
Forward/backward pass size (MB): 0.05
Params size (MB): 0.25
Estimated Total Size (MB): 0.30


#### VGG-11

In [13]:
# TODO

## Export models to ONNX

In [14]:
def export(model: Model, path: Optional[Path] = None):
    input_ = torch.rand(1, *model.input_shape)
    torch.onnx.export(
        model.model,  # Actual keras.Model object
        input_,
        str(path or model.onnx_path),
        export_params=True,
        opset_version=10,
        do_constant_folding=True,
        input_names=["input"],
        output_names=["output"],
        dynamic_axes={
            "input": {0: "batch_size"},
            "output": {0: "batch_size"},
        }
    )

In [15]:
for model in models.values():
    export(model)

## Generate proofs using `ezkl`

In [16]:
def torch_tensor_to_list(tensor: torch.Tensor) -> list[float]:
    return (tensor.detach().numpy()).reshape([-1]).tolist()

In [17]:
def random_input_data(samples: int, *shape: int, scale: int = 1) -> dict[Literal["input_data"], list[list[float]]]:
    data_list = torch_tensor_to_list(
        torch.randn(samples, *shape, requires_grad=True) * scale
    )
    return {"input_data": [data_list]}

In [18]:
for model in models.values():

    # FIXME Remove when executing code for real
    if model.name != ModelAttributes.MNIST.model_name:
        continue

    print("#"*20)
    print(model.name)
    print("#"*20)

    py_run_args = ezkl.PyRunArgs()
    py_run_args.input_visibility = "private"
    py_run_args.output_visibility = "public"
    py_run_args.param_visibility = "fixed"  # private by default

    ezkl.gen_settings(
        model.onnx_path, model.output_dir / "settings.json", py_run_args=py_run_args
    )

    if not (path := model.calibration_data_path).exists():
        with path.open("w") as f:
            json.dump(random_input_data(20, *model.input_shape), f)
    await ezkl.calibrate_settings(
        model.calibration_data_path,
        model.onnx_path,
        model.output_dir / "settings.json",
        "resources",
    )

    ezkl.compile_circuit(
        model.onnx_path,
        model.output_dir / "compiled",
        model.output_dir / "settings.json",
    )

    await ezkl.get_srs(model.output_dir / "settings.json")


    if not (path := model.inference_data_path).exists():
        with path.open("w") as f:
            json.dump(random_input_data(1, *model.input_shape), f)
    await ezkl.gen_witness(
        model.inference_data_path,
        model.output_dir / "compiled",
        model.output_dir / "witness.json",
    )
    ezkl.setup(
        model.output_dir / "compiled",
        model.output_dir / "vk",
        model.output_dir / "pk",
    )

    ezkl.prove(
        model.output_dir / "witness.json",
        model.output_dir / "compiled",
        model.output_dir / "pk",
        model.output_dir / "proof",
        "single",
    )

    is_verified = ezkl.verify(
        model.output_dir / "proof",
        model.output_dir / "settings.json",
        model.output_dir / "vk",
    )
    if is_verified:
        print("VERIFIED!")

####################
mnist
####################


circuit creation from run args failed: CircuitError(TensorError(SigBitTruncationError))
circuit creation from run args failed: CircuitError(TensorError(SigBitTruncationError))
circuit creation from run args failed: CircuitError(TensorError(SigBitTruncationError))
circuit creation from run args failed: CircuitError(TensorError(SigBitTruncationError))
circuit creation from run args failed: CircuitError(TensorError(SigBitTruncationError))
circuit creation from run args failed: CircuitError(TensorError(SigBitTruncationError))


## Load ONNX model to generate output

In [12]:
output_model = models["lenet5"]
output_model

Model(name='lenet5', input_shape=(1, 32, 32), model=Lenet5(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (classifier): Sequential(
    (0): Linear(in_features=400, out_features=120, bias=True)
    (1): Linear(in_features=120, out_features=84, bias=True)
    (2): Linear(in_features=84, out_features=10, bias=True)
  )
), root=PosixPath('/home/tomaz/git/Politecnico/Projects/zkp-vml-research/my-work/ezkl'), onnx_path=PosixPath('/home/tomaz/git/Politecnico/Projects/zkp-vml-research/my-work/ezkl/models/lenet5.onnx'), calibration_data_path=PosixPath('/home/tomaz/git/Politecnico/Projects/zkp-vml-research/my-work/ezkl/data/2-calibration/lenet5.json'), inference_data_path=PosixPath('/home/tomaz/git/Politecnico/Projects/zkp-vml-research/my-work/ezkl/data/3-inference/lenet5.json'), output_dir=PosixPath('/home/tomaz/git/Politecnico/Projects/zkp-vml-research/my-work/ezk

In [31]:
onnx_model = onnx.load(output_model.onnx_path)
onnx_model

ir_version: 5
producer_name: "pytorch"
producer_version: "2.3.1"
graph {
  node {
    input: "input"
    input: "conv1.weight"
    input: "conv1.bias"
    output: "/conv1/Conv_output_0"
    name: "/conv1/Conv"
    op_type: "Conv"
    attribute {
      name: "dilations"
      ints: 1
      ints: 1
      type: INTS
    }
    attribute {
      name: "group"
      i: 1
      type: INT
    }
    attribute {
      name: "kernel_shape"
      ints: 5
      ints: 5
      type: INTS
    }
    attribute {
      name: "pads"
      ints: 0
      ints: 0
      ints: 0
      ints: 0
      type: INTS
    }
    attribute {
      name: "strides"
      ints: 1
      ints: 1
      type: INTS
    }
  }
  node {
    input: "/conv1/Conv_output_0"
    input: "/conv1/Conv_output_0"
    output: "/Mul_output_0"
    name: "/Mul"
    op_type: "Mul"
  }
  node {
    input: "/Mul_output_0"
    input: "/conv1/Conv_output_0"
    output: "/Add_output_0"
    name: "/Add"
    op_type: "Add"
  }
  node {
    input: "/Add_

In [32]:
with output_model.inference_data_path.open("r") as f:
    output_data = json.load(f)
output_data

{'input_data': [[-1.8987312316894531,
   -0.43905773758888245,
   -1.6570171117782593,
   -0.0473284013569355,
   0.7208002209663391,
   -0.04335840418934822,
   -0.7895700931549072,
   1.570343255996704,
   1.5158369541168213,
   -0.526298463344574,
   1.2532014846801758,
   -0.14652833342552185,
   1.1281245946884155,
   -0.32866591215133667,
   -1.041646957397461,
   0.6887574195861816,
   0.44377681612968445,
   -0.805374801158905,
   0.11259883642196655,
   1.4856263399124146,
   0.8450068235397339,
   1.1734473705291748,
   -1.01834237575531,
   1.0420645475387573,
   -0.6021389961242676,
   0.33111390471458435,
   -0.31320494413375854,
   0.09507530182600021,
   0.813865065574646,
   -0.35862496495246887,
   1.0222563743591309,
   0.317489892244339,
   -0.9770033955574036,
   0.6263461112976074,
   1.8520984649658203,
   0.4238309860229492,
   0.23981697857379913,
   1.108128547668457,
   -0.9195009469985962,
   -0.30055102705955505,
   1.2431151866912842,
   -0.8376864194869995

In [33]:
output_input_array = np.array(output_data["input_data"]).reshape((1, *output_model.input_shape)).astype("float32")
output_input_array.dtype

dtype('float32')

In [35]:
ort_session = ort.InferenceSession(output_model.onnx_path)
output =  ort_session.run(None, {"input": output_input_array})[0]
output

array([[-0.07969485, -0.00710773, -0.036043  ,  0.06799266,  0.06184233,
         0.13232501, -0.20319733, -0.04050026,  0.09217978,  0.02416388]],
      dtype=float32)

In [21]:
with (output_model.output_dir / "witness.json").open("r") as f:
    witness_output = np.array(
        json.load(f)
        ["pretty_elements"]
        ["rescaled_outputs"]
    ).astype("float32")
witness_output

array([[-0.07968262, -0.00720437, -0.03595561,  0.06787407,  0.0617477 ,
         0.13223216, -0.20321003, -0.04055153,  0.09224486,  0.02423722]],
      dtype=float32)

----

## Generate proof for a single test model

In [31]:
data_path = 'input.json'

model_path = "test.onnx"
settings_path = "settings.json"

compiled_model_path = 'test.compiled'

pk_path = 'test.pk'
vk_path = 'test.vk'

witness_path = 'witness.json'


In [32]:
test_model = models["small_cnn"]
test_model

Model(name='small_cnn', input_shape=(1, 10, 10), model=SmallCnn(
  (conv): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
), root=PosixPath('/home/suller/ezkl'), onnx_path=PosixPath('/home/suller/ezkl/models/small_cnn.onnx'), calibration_data_path=PosixPath('/home/suller/ezkl/data/2-calibration/small_cnn.json'), inference_data_path=PosixPath('/home/suller/ezkl/data/3-inference/small_cnn.json'), output_dir=PosixPath('/home/suller/ezkl/output/small_cnn'))

In [33]:
export(test_model, path=Path(model_path))

In [35]:
data_array = torch_tensor_to_list(torch.randn(1, *test_model.input_shape))
data = dict(input_data = [data_array])
with open(data_path, "w") as f:
    json.dump(data, f)

In [37]:
py_run_args = ezkl.PyRunArgs()
py_run_args.input_visibility = "private"
py_run_args.output_visibility = "public"
py_run_args.param_visibility = "fixed" # private by default

res = ezkl.gen_settings(model_path, settings_path, py_run_args=py_run_args)

In [38]:
cal_path = os.path.join("calibration.json")

data_array = (torch.rand(20, *test_model.input_shape, requires_grad=True).detach().numpy()).reshape([-1]).tolist()

data = dict(input_data = [data_array])

# Serialize data into file:
json.dump(data, open(cal_path, 'w'))


await ezkl.calibrate_settings(cal_path, model_path, settings_path, "resources")

Using 2 columns for non-linearity table.
Using 3 columns for non-linearity table.
Using 3 columns for non-linearity table.
Using 2 columns for non-linearity table.
Using 3 columns for non-linearity table.
Using 3 columns for non-linearity table.
Using 3 columns for non-linearity table.
Using 5 columns for non-linearity table.
Using 5 columns for non-linearity table.
Using 5 columns for non-linearity table.
Using 9 columns for non-linearity table.


 <------------- Numerical Fidelity Report (input_scale: 13, param_scale: 13, scale_input_multiplier: 10) ------------->

+-----------------+-----------------+---------------+----------------+----------------+------------------+---------------+-------------------+--------------------+--------------------+------------------------+
| mean_error      | median_error    | max_error     | min_error      | mean_abs_error | median_abs_error | max_abs_error | min_abs_error     | mean_squared_error | mean_percent_error | mean_abs_percent_error |
+-----

True

In [39]:
res = ezkl.compile_circuit(model_path, compiled_model_path, settings_path)
res

True

In [40]:
# srs path
res = await ezkl.get_srs(settings_path)
res

True

In [41]:
# now generate the witness file

res = await ezkl.gen_witness(data_path, compiled_model_path, witness_path)
assert os.path.isfile(witness_path)

In [42]:
# HERE WE SETUP THE CIRCUIT PARAMS
# WE GOT KEYS
# WE GOT CIRCUIT PARAMETERS
# EVERYTHING ANYONE HAS EVER NEEDED FOR ZK


res = ezkl.setup(
    compiled_model_path,
    vk_path,
    pk_path,
)

assert res == True
assert os.path.isfile(vk_path)
assert os.path.isfile(pk_path)
assert os.path.isfile(settings_path)

In [43]:
# GENERATE A PROOF


proof_path = os.path.join("test.pf")

res = ezkl.prove(
    witness_path,
    compiled_model_path,
    pk_path,
    proof_path,
    "single",
)

print(res)
assert os.path.isfile(proof_path)

{'instances': [['...']], 'proof': '0x0d45aae5de297da337b59c6925ff33b8c705ce4926de50b1ab84c8a44c37286d089b3532621bd567e78ab0cd40e1c0be374b3aa61fa5c1d22a159f09df65dcac2b2d6797db1af0ad7ea7e9d8039c8542467fbf9368decd2a2e9ac1005614772d3060b688324572cf943da3fb13b8f2f9f5cbd442cb3cb7eae3e5f25856aac83902db6b19d0c37520b1dbff08bdd1a24404a71b788ce9845bbb49d7616d510da9292f01a71f0005b5aaa7aadd7326d1d43ebfc25728d186b55a36cf412b4c7048243fcc221ab6f111d13459f3beb9b259d613f7d6e6fd4269f23a46ad212cbbde1cd4f17a2182becd7d5e2c4d152aaa3665ba0591ef1d7117446a04dc5bc41923135f2e3234c087b6eb75fce024dc538c6b2a6be8398277539887354b6ae8d43c10d012b0d8ee88cd570ca3b4e72617f32c5cdf6f8e74560927e55236956632bc088285d9a74e78cfc06c237366a12d25369a52306728c2f98962b166d809ec2d2c6c2317376a933d7c98fa202400b9c38e1d74e0399baec5083a06f8f71c9641098b800b470ed2c84e6e1efbe9f72e806fc62939a6be955291d5f359377edfbd18e29515cc181813811bc30c9559abd308bf88b6a5528ee295a8bed39ae71b5a07d094bca8035f5076ec125a2e419da85fac52cf99c64f42aac49d80583fcafd0be

In [44]:
# VERIFY IT

res = ezkl.verify(
    proof_path,
    settings_path,
    vk_path,
)

if res:
    print("verified")

verified
